In [21]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
data = pd.read_csv(r"C:\Users\landg\Downloads\archive\tested.csv")

# Inspect the dataset
data.head()         # Display the first few rows



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])


# Drop 'Cabin' column
data.drop(columns=['Cabin'], inplace=True)

# Verify missing values
print(data.isnull().sum())


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Embarked       0
dtype: int64


In [25]:
# Handle outliers in 'Fare'
Q1 = data['Fare'].quantile(0.25)
Q3 = data['Fare'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove rows with outliers in 'Fare'
data = data[(data['Fare'] >= lower_bound) & (data['Fare'] <= upper_bound)]

# Verify data shape
print(data.shape)


(362, 11)


In [27]:
# Convert 'PassengerId' to categorical
data['PassengerId'] = data['PassengerId'].astype('category')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [29]:
from sklearn.preprocessing import StandardScaler

# Select numerical features
numerical_features = ['Age', 'Fare']

# Normalize using StandardScaler
scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Check scaled data
print(data[numerical_features].head())


        Age      Fare
0  0.588694 -0.727059
1  1.704206 -0.784848
2  3.042820 -0.597552
3 -0.080613 -0.668986
4 -0.526817 -0.416354


In [31]:
# Feature engineering
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# One-hot encoding
data = pd.get_dummies(data, columns=['Sex', 'Embarked'], drop_first=True)

# Drop irrelevant columns
data.drop(columns=['Name', 'Ticket', 'PassengerId'], inplace=True)

# Check final dataset
data.head()


,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S
0,0,3,0.588694,0,0,-0.727059,1,True,True,False
1,1,3,1.704206,1,0,-0.784848,2,False,False,True
2,0,2,3.042820,0,0,-0.597552,1,True,True,False
3,0,3,-0.080613,0,0,-0.668986,1,True,False,True
4,1,3,-0.526817,1,1,-0.416354,3,False,False,True
